## Semantic Kernel: Ramp-Up based on SK's Documentation

To get the latest version of SK Python package, use:

``` bash
pip install --upgrade semantic-kernel
```

## 📒 Notebook 6: Plugins

### 🪜 Step 1: Configure environment

In [1]:
# Import required packages
import os
import yaml
import logging
from urllib.parse import urlparse, parse_qs

from semantic_kernel import Kernel
from semantic_kernel.contents import ChatHistory
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.functions import KernelArguments
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings

# For Azure Logic Apps management
from azure.identity import DefaultAzureCredential
from azure.mgmt.web import WebSiteManagementClient

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Set Azure OpenAI backend variables
AOAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_API_DEPLOY")
AOAI_ENDPOINT = os.getenv("AZURE_OPENAI_API_BASE")
AOAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

# Set Azure Logic Apps variables
SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID")
RESOURCE_GROUP = os.getenv("RESOURCE_GROUP_NAME", "App_LogicApp")
LOGIC_APP_NAME = "Laziz-StandardLA"     # Your Standard Logic App name
WORKFLOW_NAME = "lazizdemologicapp"     # Your workflow inside the Standard Logic App
TRIGGER_NAME = "HTTP-trigger-standard"  # Your HTTP trigger name

### 🪜 Step 2: Generate OpenAPI Spec for Logic App

In [4]:
# Define helper class for Standard Logic Apps OpenAPI spec generation
class StandardLogicAppsOpenAPIHelper:
    """
    Helper class to generate OpenAPI spec for Standard Logic Apps.
    """
    
    def __init__(self, subscription_id: str, resource_group: str, credential=None):
        if credential is None:
            credential = DefaultAzureCredential()
        
        self.subscription_id = subscription_id
        self.resource_group = resource_group
        self.credential = credential
        self.web_client = WebSiteManagementClient(credential, subscription_id)
    
    def get_callback_url(self, logic_app_name: str, workflow_name: str, trigger_name: str) -> str:
        """Get the callback URL with SAS token for a Standard Logic App workflow trigger."""
        try:
            callback = self.web_client.workflow_triggers.list_callback_url(
                resource_group_name = self.resource_group,
                name = logic_app_name,
                workflow_name = workflow_name,
                trigger_name = trigger_name
            )
            if callback.value is None:
                raise ValueError(f"No callback URL returned for workflow '{workflow_name}' in Logic App '{logic_app_name}'.")
            print(f"Retrieved callback URL for workflow: {workflow_name}")
            return callback.value
        except Exception as e:
            print(f"Error getting callback URL: {e}")
            raise
    
    def create_openapi_spec(self, callback_url: str, operation_description: str = None) -> str:
        """
        Create OpenAPI 3.0 spec with explicit SAS parameters for direct invocation.
        """
        if operation_description is None:
            operation_description = "Get weather forecast for a specified location"
        from urllib.parse import urlparse, parse_qs
        parsed_url = urlparse(callback_url)
        base_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
        query_params = parse_qs(parsed_url.query)
        parameters_yaml = []
        for param_name, param_values in query_params.items():
            if param_values:
                param_value = param_values[0]
                param_entry = f"""        - name: {param_name}
          in: query
          required: true
          schema:
            type: string
            default: '{param_value}'"""
                parameters_yaml.append(param_entry)
        parameters_section = "\n".join(parameters_yaml)
        openapi_yaml = f"""openapi: 3.0.0
info:
  title: Standard Logic App Weather API
  version: 1.0.0
  description: Weather forecast service via Azure Standard Logic App
servers:
  - url: {base_url}
paths:
  /:
    post:
      operationId: getWeatherForecast
      summary: Get weather forecast
      description: {operation_description}
      parameters:
{parameters_section}
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                Location:
                  type: string
                  description: The location to get weather forecast for
              required:
                - Location
      responses:
        '200':
          description: Successful response
"""
        print("OpenAPI spec with auth parameters created successfully")
        return openapi_yaml

print("StandardLogicAppsOpenAPIHelper class defined")

StandardLogicAppsOpenAPIHelper class defined


In [5]:
# Initialise the helper
logic_helper = StandardLogicAppsOpenAPIHelper(
    subscription_id = SUBSCRIPTION_ID,
    resource_group = RESOURCE_GROUP
)

# Get callback URL
callback_url = logic_helper.get_callback_url(
    logic_app_name = LOGIC_APP_NAME,
    workflow_name = WORKFLOW_NAME,
    trigger_name = TRIGGER_NAME
)

# Generate OpenAPI spec
openapi_spec = logic_helper.create_openapi_spec(
    callback_url = callback_url,
    operation_description = "Get current weather forecast for any location worldwide"
)

print("\n" + "="*70)
print("OpenAPI Specification Preview:")
print("="*70)
print(openapi_spec[:70] + "...")

ValueError: Parameter 'subscription_id' must not be None.

### 🪜 Step 3: Add Logic App's OpenAPI plug-in

In [ ]:
# Initialise kernel
kernel = Kernel()

In [ ]:
# Configure logging
logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

In [ ]:
# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    deployment_name = AOAI_DEPLOYMENT,
    endpoint = AOAI_ENDPOINT,
    api_version = AOAI_API_VERSION,
    service_id = "azure_openai_chat",
)

kernel.add_service(chat_completion)

In [ ]:
# Add the OpenAPI plugin to the kernel directly from parsed dict (no file needed)
try:
    # Parse the YAML spec into a dictionary
    openapi_dict = yaml.safe_load(openapi_spec)
    print("OpenAPI spec parsed into dictionary")
    
    # Import the plugin directly from the parsed dictionary
    plugin = kernel.add_plugin_from_openapi(
        plugin_name = "WeatherPlugin",
        openapi_parsed_spec = openapi_dict
    )

    print("OpenAPI plugin 'WeatherPlugin' added to kernel successfully")
    print(f"Available functions: {list(plugin.functions.keys())}")
    
except Exception as e:
    print(f"Error adding OpenAPI plugin: {e}")

OpenAPI spec parsed into dictionary
OpenAPI plugin 'WeatherPlugin' added to kernel successfully
Available functions: ['getWeatherForecast']


### 🪜 Step 4: Test direct plug-in's function call

In [ ]:
# Test the plugin by calling the function directly
try:
    # Get the function from the plugin
    weather_function = kernel.plugins["WeatherPlugin"]["getWeatherForecast"]
    
    # Parse the SAS tokens from the callback URL and normalize their names
    parsed_url = urlparse(callback_url)
    query_params = parse_qs(parsed_url.query)

    # Create arguments, replacing hyphens with underscores in the SAS token names
    arguments = KernelArguments(
        Location = "London",
        **{k.replace("-", "_"): v[0] for k, v in query_params.items()}
    )
    
    # Invoke the function with all required arguments
    result = await weather_function.invoke(kernel, arguments)
    
    print("="*70)
    print("Direct Function Call Result:")
    print("="*70)
    print(f"Location: London")
    print(f"Response: {result}")
    
except Exception as e:
    print(f"Error invoking function: {e}")

Direct Function Call Result:
Location: London
Response: {"responses":{"daily":{"day":{"cap":"Partly sunny","pvdrCap":"Partly sunny","pvdrWindDir":"246","pvdrWindSpd":"16","icon":3,"symbol":"d2000","pvdrIcon":"3","urlIcon":"http://img-s-msn-com.akamaized.net/tenant/amp/entityid/AAehLNN.img","precip":10.0,"wx":"-RA","sky":"SCT","windDir":246,"windSpd":16.0,"summary":"Expect partly sunny skies.  The high will be 13°.","summaries":["Expect partly sunny skies."," The high will be 13°."]},"night":{"cap":"Light rain showers","pvdrCap":"Light rain showers","pvdrWindDir":"219","pvdrWindSpd":"15","icon":46,"symbol":"n3100","pvdrIcon":"46","urlIcon":"http://img-s-msn-com.akamaized.net/tenant/amp/entityid/AAehyQD.img","precip":52.0,"wx":"-RA","sky":"SCT","windDir":219,"windSpd":15.0,"summary":"There will be scattered light rain showers.  The low will be 8°.","summaries":["There will be scattered light rain showers."," The low will be 8°."]},"pvdrCap":"Partly sunny","pvdrWindDir":"246","pvdrWindSpd

### 🪜 Step 5: The the full multi-turn conversation

In [ ]:
# Create execution settings with auto function calling enabled
execution_settings = OpenAIChatPromptExecutionSettings(
    service_id = "azure_openai_chat",
    max_tokens = 2000,
    temperature = 0.7,
    function_choice_behavior = FunctionChoiceBehavior.Auto()
)

In [ ]:
# Create a chat history
chat_history = ChatHistory()
chat_history.add_system_message(
    "You are a helpful weather assistant. When asked about weather, "
    "use the getWeatherForecast function to get current weather information."
)

# Get the chat completion service
chat_service = kernel.get_service("azure_openai_chat")

In [ ]:
# Helper function to run a chat completion with function calling
async def run_chat_completion(query: str):
    try:
        chat_history.add_user_message(query)
    
        # Get response with function calling enabled
        response = await chat_service.get_chat_message_contents(
            chat_history = chat_history,
            settings = execution_settings,
            kernel = kernel,
            arguments = KernelArguments()
        )
        
        print("Chat Response:")
        print("="*70)
        print(f"User: {query}")
        
        # Add ALL response messages to history
        for msg in response:
            chat_history.messages.append(msg)
            # Only print the final assistant message
            if msg.role.value == "assistant" and msg.content:
                 print(f"\nAI Agent: {msg.content}")

    except Exception as e:
        print(f"Error during chat completion: {e}")

In [ ]:
# Request weather info for Paris
query1 = "What's the weather like in Paris?"
await run_chat_completion(query1)

Chat Response:
User: What's the weather like in Paris?

AI Agent: The weather in Paris is partly sunny with a high of 14°C during the day. There is a 22% chance of precipitation and winds are coming from the west-southwest at 14 km/h. At night, it will be mostly cloudy with a low of 7°C and winds at 9 km/h.


In [ ]:
# Request weather info for Tokyo
query2 = "How about Tokyo?"
await run_chat_completion(query2)

Chat Response:
User: How about Tokyo?

AI Agent: The weather in Tokyo is mostly sunny with a high of 19°C during the day. There is a very low chance of precipitation at 3%, and winds are coming from the northwest at 23 km/h. At night, the skies will be mostly clear with a low of 10°C and winds at 19 km/h.


In [ ]:
# Display the full conversation
print("\nFull Conversation History:")
print("="*70)

for i, message in enumerate(chat_history.messages, 1):
    role = message.role.value if hasattr(message.role, 'value') else str(message.role)
    role = role.capitalize()
    
    print(f"\n[{i}] {role}:")
    
    # Check for content
    if hasattr(message, 'content') and message.content:
        print(f"  {message.content}")
    
    # Check for function calls
    if hasattr(message, 'items') and message.items:
        for item in message.items:
            if hasattr(item, 'function_name'):
                print(f"     - Function Call: {item.function_name}")
                if hasattr(item, 'arguments'):
                    print(f"     - Arguments: {str(item.arguments)[:50]}...")
            if hasattr(item, 'result'):
                print(f"     - Function Result: {str(item.result)[:50]}...")
    
    print("-"*70)

print(f"\nTotal messages in history: {len(chat_history.messages)}")


Full Conversation History:

[1] System:
  You are a helpful weather assistant. When asked about weather, use the getWeatherForecast function to get current weather information.
----------------------------------------------------------------------

[2] User:
  What's the weather like in Paris?
----------------------------------------------------------------------

[3] Assistant:
     - Function Call: getWeatherForecast
     - Arguments: {"api_version":"2022-05-01","sp":"/triggers/HTTP-t...
----------------------------------------------------------------------

[4] Tool:
     - Function Call: getWeatherForecast
     - Function Result: {"responses":{"daily":{"day":{"cap":"Partly sunny"...
----------------------------------------------------------------------

[5] Assistant:
  The weather in Paris is partly sunny with a high of 14°C during the day. There is a 22% chance of precipitation and winds are coming from the west-southwest at 14 km/h. At night, it will be mostly cloudy with a low